In [7]:
zips = !find ./gutenberg/ -name '*.zip' 
zips

['./gutenberg/aleph.gutenberg.org/1/2/3/8/12386/12386.zip',
 './gutenberg/aleph.gutenberg.org/1/2/3/8/12383/12383-8.zip',
 './gutenberg/aleph.gutenberg.org/1/2/3/8/12383/12383.zip',
 './gutenberg/aleph.gutenberg.org/1/2/3/8/12385/12385.zip',
 './gutenberg/aleph.gutenberg.org/1/2/3/8/12385/12385-8.zip',
 './gutenberg/aleph.gutenberg.org/1/2/3/8/12380/12380-8.zip',
 './gutenberg/aleph.gutenberg.org/1/2/3/8/12380/12380.zip',
 './gutenberg/aleph.gutenberg.org/1/2/3/8/12381/12381.zip',
 './gutenberg/aleph.gutenberg.org/1/2/3/7/12377/12377.zip',
 './gutenberg/aleph.gutenberg.org/1/2/3/7/12373/12373.zip',
 './gutenberg/aleph.gutenberg.org/1/2/3/7/12373/12373-8.zip',
 './gutenberg/aleph.gutenberg.org/1/2/3/7/12370/12370.zip',
 './gutenberg/aleph.gutenberg.org/1/2/3/7/12370/12370-8.zip',
 './gutenberg/aleph.gutenberg.org/1/2/3/7/12374/12374-8.zip',
 './gutenberg/aleph.gutenberg.org/1/2/3/7/12374/12374.zip',
 './gutenberg/aleph.gutenberg.org/1/2/3/7/12372/12372-8.zip',
 './gutenberg/aleph.gutenb

In [19]:
import zipfile
import re
import webdataset as wds

In [71]:
!rm -f gutenshards/*.tar
zips = !find ./gutenberg/ -name '*.zip' 
count = 0
writer = wds.ShardWriter("gutenshards/gutenberg-%06d.tar", maxcount=200, maxsize=100e6)
for zipname in zips:
    zf = zipfile.ZipFile(zipname, 'r')
    for txtname in zf.namelist():
        if txtname.endswith('.txt'):
            data = zf.read(txtname)
            base = txtname[:-len('.txt')]
            # print(zipname+"/"+txtname, len(data))
            writer.write({"__key__": zipname+"/"+base, "txt": data})
            count += 1
print(count)


# writing gutenshards/gutenberg-000000.tar 0 0.0 GB 0
# writing gutenshards/gutenberg-000001.tar 200 0.1 GB 200
# writing gutenshards/gutenberg-000002.tar 200 0.1 GB 400
# writing gutenshards/gutenberg-000003.tar 200 0.1 GB 600
# writing gutenshards/gutenberg-000004.tar 200 0.1 GB 800
# writing gutenshards/gutenberg-000005.tar 200 0.1 GB 1000
# writing gutenshards/gutenberg-000006.tar 200 0.1 GB 1200
1335


In [72]:
shards = !find gutenshards/. -name '*.tar'
len(shards)

7

In [73]:
import re

def cut_text_between_markers(text, start=r"\*\*\* START.*\n", end="\*\*\* END.*\n"):
    start_index = re.search(start, text).end()
    end_index = re.search(end, text).start()
    if start_index == -1 or end_index == -1:
        return None
    return text[start_index:end_index].strip()

In [76]:
ds = wds.WebDataset("gutenshards/gutenberg-{000000..000006}.tar").decode()
for sample in ds:
    txt = sample["txt"]
    content = cut_text_between_markers(txt)
    if content is None:
        print("skipping", sample["__key__"])

UnicodeDecodeError: 'utf-8' codec can't decode byte 0xea in position 12494: invalid continuation byte

In [37]:
sample.keys()

dict_keys(['__key__', '__url__', 'txt.gz'])

In [55]:
"*** START" in txt

True